In [1]:
import numpy as np
import os
import tensorflow as tf
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from PIL import Image
import numpy as np

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
labels = []
list_imgs = []
for element_type in os.listdir('./resized_img//'):
    for pic in os.listdir('./resized_img//' + element_type):
        try:
            img = Image.open('./resized_img/' + element_type + '/' + pic)
            img = np.array(img).flatten()
            labels.append(element_type)
            list_imgs.append(img)
        except IsADirectoryError:
            print(".ipynb_checkpoints error")

In [3]:
images = np.array(list_imgs)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train = X_train.astype("float") / 255
X_test = X_test.astype("float") / 255

In [15]:
X_train = X_train.reshape(-1, 128, 128, 3)
X_test = X_test.reshape(-1, 128, 128, 3)

In [16]:
lb = LabelBinarizer()
y_train_OHE = lb.fit_transform(y_train)
y_test_OHE = lb.transform(y_test)

In [7]:
X_train.shape

(1697, 128, 128, 3)

# convnet

In [8]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(128,128,3)))
model.add(tf.keras.layers.MaxPool2D(pool_size=(5, 5)))
#model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
#model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3)))
model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(128, activation='relu'))
#model.add(tf.keras.layers.Dense(12, activation='softmax'))
model.add(tf.keras.layers.Dense(12))
model.add(tf.keras.layers.Activation("softmax"))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 12)                221196    
_________________________________________________________________
activation (Activation)      (None, 12)                0         
Total params: 223,628
Trainable params: 223,628
Non-trainable params: 0
_________________________________________________________________


In [10]:
epochs = 10
batch_size=128

In [12]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(lr=0.01),
              metrics=['accuracy'])

model.fit(X_train, y_train_OHE,
          batch_size=batch_size,
          epochs=epochs,
          verbose=True,
          validation_data=(X_test, y_test_OHE))
score = model.evaluate(X_test, y_test_OHE, verbose=True)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 1697 samples, validate on 425 samples
Epoch 1/10
1697/1697 [==============================] - 38s 22ms/step - loss: 14.6839 - acc: 0.0890 - val_loss: 14.4873 - val_acc: 0.1012
Epoch 2/10
1697/1697 [==============================] - 38s 23ms/step - loss: 14.6839 - acc: 0.0890 - val_loss: 14.4873 - val_acc: 0.1012
Epoch 3/10
 896/1697 [==============>...............] - ETA: 16s - loss: 14.5531 - acc: 0.0971

KeyboardInterrupt: 

0.0